# Benchmark Summary Statistics
---

Description:

    This notebook performs the summary statisticks results for all benchmark functions (one at a time).
    The optimization process is repeated N=50 times (with default settings) and then we collect the results.
    
---

### Import python libraries and set up the project directory

In [6]:
import os, sys
import numpy as np

PROJECT_DIR = os.path.abspath('..')
sys.path.append(PROJECT_DIR)

### Import the custom PSO code

In [7]:
from star_pso.population.swarm import Swarm
from star_pso.population.particle import Particle
from star_pso.engines.standard_pso import StandardPSO
from star_pso.utils.auxiliary import cost_function

### Load all the Benchmark problems

In [8]:
# Dimensions: 1D
from star_pso.benchmarks.equal_maxima import EqualMaxima
from star_pso.benchmarks.five_uneven_peak_trap import FiveUnevenPeakTrap
from star_pso.benchmarks.uneven_decreasing_maxima import UnevenDecreasingMaxima

# Dimensions: 2D
from star_pso.benchmarks.himmelblau import Himmelblau
from star_pso.benchmarks.gaussian_mixture import GaussianMixture
from star_pso.benchmarks.six_hump_camel_back import SixHumpCamelBack

# Dimensions: N >= 2D
from star_pso.benchmarks.shubert import Shubert
from star_pso.benchmarks.vincent import Vincent
from star_pso.benchmarks.rastrigin import Rastrigin

In [9]:
# Select one test problem.
benchmark = GaussianMixture()

### Define the cost function

Note: All the optimization problems are solved as maximizations.

In [10]:
@cost_function(minimize=False)
def func_benchmark(x_array: np.ndarray, **kwargs) -> float:
    # Return the solution.
    return benchmark.func(x_array).item()
# _end_def_

### Setup the experiment

In [11]:
# Number of repeatitions.
n_repeats = 50

# Swarm population size.
POP_SIZE = 85

# Maximum function evaluations.
MAX_F_EVALS = 50_000

# Store the results.
results = n_repeats * [0]

# Display Info.
print(f"Testing function: {benchmark.name}\n")

# Go time ...
for i in range(n_repeats):

    # Display current iteration.
    print(f"Running optimization: {i+1} out of {n_repeats} ... ", flush=True)

    # Get an initial (new) sample.
    x_sample = benchmark.sample_random_positions(n_pos=POP_SIZE)

    # Initial swarm population.
    swarm_t0 = Swarm([Particle(x) for x in x_sample])
    
    # Create the StandardPSO object that will carry on the optimization.
    test_PSO = StandardPSO(initial_swarm = swarm_t0, obj_func = func_benchmark,
                           x_min = benchmark.x_min, x_max = benchmark.x_max)
    # Run the optimization.
    test_PSO.run(max_it = 800, f_max_eval = MAX_F_EVALS, verbose = False,
                 options = {"w0": 0.70, "c1": 1.50, "c2": 1.50, "mode": "multimodal"})
    
    # Obtain the number of the global optima that were found for epsilon = 1.0e-4.
    gopt_found, gopt_total = benchmark.search_for_optima(test_PSO.swarm.best_n(POP_SIZE))

    # Update the results.
    results[i] = gopt_found

    # Blank line to declatter the screen.
    print(" ")
# _end_for_

print("Done.")

Testing function: GaussianMixture

Running optimization: 1 out of 50 ... 


11/30/2025 16:41:14 INFO: Initial f_optimal = -2.4162
11/30/2025 16:41:28 WARNING: StandardPSO reached the maximum number of function evaluations at iteration 588


Final f_optimal = -1.8329
run: elapsed time = 14.358 seconds.
 
Running optimization: 2 out of 50 ... 


11/30/2025 16:41:28 INFO: Initial f_optimal = -2.0362
11/30/2025 16:41:37 WARNING: StandardPSO reached the maximum number of function evaluations at iteration 588


Final f_optimal = -1.8329
run: elapsed time = 8.631 seconds.
 
Running optimization: 3 out of 50 ... 


...

Final f_optimal = -1.8329
run: elapsed time = 10.207 seconds.
 
Running optimization: 49 out of 50 ... 


11/30/2025 16:48:55 INFO: Initial f_optimal = -3.1913
11/30/2025 16:49:04 WARNING: StandardPSO reached the maximum number of function evaluations at iteration 588


Final f_optimal = -1.8329
run: elapsed time = 9.406 seconds.
 
Running optimization: 50 out of 50 ... 


11/30/2025 16:49:04 INFO: Initial f_optimal = -2.3512
11/30/2025 16:49:13 WARNING: StandardPSO reached the maximum number of function evaluations at iteration 588


Final f_optimal = -1.8329
run: elapsed time = 8.992 seconds.
 
Done.


In [12]:
# Compute the "Peak Ratio".
peak_ratio = sum(results) / (gopt_total * n_repeats)

print(f"PR = {peak_ratio}")

PR = 1.0


In [13]:
# Compute the "Success Rate".
success_rate = sum([r == gopt_total for r in results]) / n_repeats

print(f"SR = {success_rate}")

SR = 1.0


### Optimization Results

Accuracy level: $\epsilon = 1.0^{-4}$

| Benchmark Function | Dimensions | No. Global opt. | MaxFEs  | Population Size | Parameters | PR | SR |
|:---|:---|:---|:---|:---|:---|:---|:---|
| Equal Maxima | 1 | 5 | 50_000| 120 | (0.70, 1.50, 1.50) | 1.0 | 1.0 |
| Five Uneven Peak Trap | 1 | 2 | 50_000 | 120 | (0.70, 1.50, 1.50) | 1.0 | 1.0 |
| Uneven Decreasing Maxima | 1 | 1 | 50_000 | 85 | (0.70, 1.50, 1.50) | 1.0 | 1.0 |
| Gaussian Mixture | 2 | 2 | 50_000 | 85 | (0.70, 1.50, 1.50) | 1.0 | 1.0 |
| Six Hump Camel Back | 2 | 2 | 50_000 | 85 | (0.70, 1.50, 1.50) |1.0 | 1.0 |
| Himmelblau | 2 | 4 | 50_000 | 100 | (0.70, 1.50, 1.50) |1.0 | 1.0 |
| Rastrigin | 2 / 4| 6 / 12 | 100_000 / 200_000 | 150 / 400 | (0.70, 1.50, 1.50) |1.000 / 0.998 | 1.000 / 0.980 |
| Shubert | 2 / 3 | 18 / 81 | 200_000 / 400_000 | 600 / 1600 | (0.75, 1.50, 1.50) |0.983 / 0.755 | 0.740 / 0.000 |
| Vincent | 2 / 3 | 36 / 216 | 200_000 / 400_000 | 2000 / 4000 | (0.70, 1.50, 1.50) |0.744 / 0.392 | 0.000 / 0.000 |

### End of file